### Introduction:


This study involves downloading a CSV file containing data on funded companies from Crunchbase. The process of data cleaning and transformation includes:

* Exploring the data and dealing with null values,
* Renaming columns for clarity,
* Creating new identifier column,
* Writing functions to identify and address areas that need attention.

### Install & Import Necessary Libraries


In [1]:
# pip install ipython-sql pymysql plotly

In [2]:

import pandas as pd


### Collecting the data


In [3]:


#extract the csv data file into a pandas dataframe

data = pd.read_csv('/Users/mariamadeoti/Downloads/companies.csv')
data.head(2)



,id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,...,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
0,c:1,0,Company,1,NaN,Wetpaint,wetpaint,/company/wetpaint,web,operating,...,2010-09-05,2013-09-18,5.0,17.0,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00,47.606209,-122.332071,15.5
1,c:10,1,Company,10,NaN,Flektor,flektor,/company/flektor,games_video,acquired,...,NaN,NaN,NaN,6.0,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14,34.021122,-118.396467,NaN


### Data Pre-processing and Cleaning


### a. select relevant columns


In [4]:

#checks the count of rows and columns in the dataframe

data.shape


(196553, 44)

In [5]:


#selects only relevant columns

required_columns = ['permalink', 'name', 'homepage_url', 'category_code', 
                    'funding_total_usd', 'status', 'country_code', 'state_code',
                       'region', 'city', 'funding_rounds', 'founded_at']


#stores the relevant columns back to the dataframe

data = data[required_columns]


#displays the first 2 rows of the dataframe

data.head(2)



,permalink,name,homepage_url,category_code,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at
0,/company/wetpaint,Wetpaint,http://wetpaint-inc.com,web,39750000.0,operating,USA,WA,Seattle,Seattle,3.0,2005-10-17
1,/company/flektor,Flektor,http://www.flektor.com,games_video,NaN,acquired,USA,CA,Los Angeles,Culver City,NaN,NaN



### b. check for duplicates


In [6]:

#check for the count of duplicates

print(f"The number of duplicate values is {data.duplicated().sum()}")


The number of duplicate values is 38


In [7]:

#delete all the duplicate rows

data.drop_duplicates(inplace=True)



In [8]:
#check for the count of duplicates again

print(f"The number of duplicate values is {data.duplicated().sum()}")


The number of duplicate values is 0



### c. check for data type consistency


In [9]:

data.dtypes


permalink             object
name                  object
homepage_url          object
category_code         object
funding_total_usd    float64
status                object
country_code          object
state_code            object
region                object
city                  object
funding_rounds       float64
founded_at            object
dtype: object

### d. check for null values

In [10]:


data.isnull().sum()



permalink                 0
name                     23
homepage_url          69976
category_code         73334
funding_total_usd    168641
status                    0
country_code         108527
state_code           145613
region                    0
city                 112627
funding_rounds       164808
founded_at           105292
dtype: int64

### e. Dropping columns - my analysis will center around funded companies, I will drop all null cells in funding total column

In [11]:


print(f"The number of null cells in the total funding usd column is {data['funding_total_usd'].isnull().sum()}")


The number of null cells in the total funding usd column is 168641


In [12]:

#drop the null cells

data.dropna(subset = ['funding_total_usd'], inplace=True)



In [13]:

#check if the null cells have been removed

print(f"The number of null cells in the total funding usd column is {data['funding_total_usd'].isnull().sum()}")


The number of null cells in the total funding usd column is 0


### f. create a unique identifier for each row

In [14]:


data.insert(data.columns.get_loc('permalink'), 'id', range(1, len(data)+1))

data.head(2)


,id,permalink,name,homepage_url,category_code,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at
0,1,/company/wetpaint,Wetpaint,http://wetpaint-inc.com,web,39750000.0,operating,USA,WA,Seattle,Seattle,3.0,2005-10-17
13,2,/company/friendfeed,FriendFeed,http://friendfeed.com,web,5000000.0,acquired,USA,CA,SF Bay,Mountain View,1.0,2007-10-01


### g. convert the name column to lowercase


In [15]:


data[['name', 'region', 'city']] = data[['name', 'region', 'city']].apply(lambda x : x.str.lower())

data.head(2)


,id,permalink,name,homepage_url,category_code,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at
0,1,/company/wetpaint,wetpaint,http://wetpaint-inc.com,web,39750000.0,operating,USA,WA,seattle,seattle,3.0,2005-10-17
13,2,/company/friendfeed,friendfeed,http://friendfeed.com,web,5000000.0,acquired,USA,CA,sf bay,mountain view,1.0,2007-10-01


### h. rename the category column


In [16]:

data.rename(columns = {'category_code':'sector'}, inplace = True)
data.head(2)

,id,permalink,name,homepage_url,sector,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at
0,1,/company/wetpaint,wetpaint,http://wetpaint-inc.com,web,39750000.0,operating,USA,WA,seattle,seattle,3.0,2005-10-17
13,2,/company/friendfeed,friendfeed,http://friendfeed.com,web,5000000.0,acquired,USA,CA,sf bay,mountain view,1.0,2007-10-01


## Store the cleaned dataframe into a new csv file

In [17]:
data.to_csv('cleaned_data.csv', index=False )